下载李宏毅2017深度学习课程资料，网址：<http://speech.ee.ntu.edu.tw/~tlkagk/courses_MLDS17.html>

In [94]:
import requests
import re
from bs4 import BeautifulSoup
import os

headers ={
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Host': 'speech.ee.ntu.edu.tw',
    'If-Modified-Since': 'Fri, 30 Nov 2018 01:14:57 GMT',
    'If-None-Match': '"1526-57bd787132c18"',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Mobile Safari/537.36'
}

res = requests.get('http://speech.ee.ntu.edu.tw/~tlkagk/courses_MLDS17.html', headers=headers)
if res.status_code != 200:
    print("网址请求失败，正重新发起请求……")
    res = requests.get('http://speech.ee.ntu.edu.tw/~tlkagk/courses_MLDS17.html', headers=headers)
print("网址请求成功！")
text = re.sub(r'<!--.*?-->', '', res.text, flags=re.S) # 去掉注释

re_dirname = re.compile(r'<li>(.*?)<a', flags=re.S)
re_link = re.compile(r'href="(.*?)">')
re_ill_char = re.compile(r'[\\/:*?"<>|]') # 去掉会导致非法文件名的字符

soup = BeautifulSoup(text, 'lxml')
items = soup.select('li')

num = 1
link_single = []
for item in items:
    item = str(item)
    dirname = re.findall(re_dirname, item)
    if not dirname:continue
    dirname = str(num) + '_' + re.sub(re_ill_char, ' ', dirname[0].strip()).strip() # 处理掉非法字符
    dirname = re.findall(re.compile(r'[^\n\r]+'), dirname)[0].strip() # 去掉包括\r或者\n之后的所有字符
    dirname = dirname[:70] # 防止字符过长
    
    try:
        os.mkdir(dirname)
    except:
        dirname = re.sub(r'\([/\d]+.*', '', dirname, flags=re.S).strip()
        os.mkdir(dirname)
        
    links = re.findall(re_link, item)
    
    down_judge = False # 条目中是否有可供下载链接判断
    
    for link in links: 
        # 提取文件类型后缀名
        suffix = ''
        for s in link[::-1]:
            if s == '.':break
            suffix = s + suffix
            
        if suffix not in ['pdf', 'pptx', 'mp4']:continue # 不是下载链接的条目不计
        down_judge = True
        
        if link not in link_single:
            link_single.append(link)
        else:
            down_judge = False
            continue
            
        res_link = requests.get('http://speech.ee.ntu.edu.tw/~tlkagk/'+link)
        while res_link.status_code != 200:
            print("第%d节课链接请求失败，正重新发起请求……" % num)
            res_link = requests.get('http://speech.ee.ntu.edu.tw/~tlkagk/'+link)
        with open(dirname+'/'+link.split('/')[-1], 'wb') as f:
            f.write(res_link.content)
    
    if down_judge:
        print("第%d节课资料下载完成" % num)
        num += 1
    else:
        os.removedirs(dirname)

网址请求成功！
第1节课资料下载完成
第2节课资料下载完成
第3节课资料下载完成
第4节课资料下载完成
第5节课资料下载完成
第6节课资料下载完成
第7节课资料下载完成
第8节课资料下载完成
第9节课资料下载完成
